In [39]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common import exceptions  
from time import sleep
from unidecode import unidecode
from random import randint

In [40]:
drive_path = './Driver/chromedriver.exe'
url = 'https://www.agoda.com/search?area=571269&locale=en-us&ckuid=5edb8609-7640-481d-bc3c-8e003673c7bf&prid=0&currency=PHP&correlationId=ce0d70a1-0b41-441b-a8c4-7653f5367d9c&analyticsSessionId=6832091592958727653&pageTypeId=103&realLanguageId=1&languageId=1&origin=PH&cid=1844104&userId=5edb8609-7640-481d-bc3c-8e003673c7bf&whitelabelid=1&loginLvl=0&storefrontId=3&currencyId=18&currencyCode=PHP&htmlLanguage=en-us&cultureInfoName=en-us&machineName=sg-pc-6g-acm-web-user-84b57d4c78-t7jm9&trafficGroupId=1&sessionId=0jcltzjj2cqf4knqndmcx4rb&trafficSubGroupId=84&aid=130589&useFullPageLogin=true&cttp=4&isRealUser=true&mode=production&browserFamily=Chrome&checkIn=2023-07-24&checkOut=2023-07-25&rooms=1&adults=2&children=0&priceCur=PHP&los=1&textToSearch=Bonifacio%20Global%20City%20(BGC)%20%2F%20Taguig&travellerType=1&familyMode=off&productType=-1'
driver = webdriver.Chrome(drive_path)

driver.get(url)

C:\Users\ryana\AppData\Local\Temp\ipykernel_18248\1940692324.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(drive_path)


In [41]:
def scrollDown():
    lastHeight = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(0.5)
        newHeight = driver.execute_script("return document.body.scrollHeight")
        if newHeight == lastHeight:
            break
        lastHeight = newHeight

def getHotelLinks():
    
    scrollDown()
    sleep(7)
    links = driver.find_elements(By.XPATH, "//a[@class = 'PropertyCard__Link']")
    sleep(5)
    
    for i in range(len(links)):
        links[i] = links[i].get_property(name = 'href')

    try:
        next_button = driver.find_element(By.XPATH, "//button[@id = 'paginationNext']")
    except(exceptions.NoSuchElementException):
        next_button = None

    return (next_button, links)

In [42]:
links = []

while True:
    sleep(5)
    args = getHotelLinks()
    links = links + args[1]    
    if args[0] == None:
        break
    else:
        args[0].click()

In [43]:
len(links)

289

In [44]:
def getHotelFeaturesDataFrame():
    hotelID = driver.find_element(By.XPATH, "//p[@class ='HeaderCerebrum__Name']").text
    features = driver.find_elements(By.XPATH,"//div[@data-element-name ='abouthotel-amenities-facilities']//div[@class = 'Box-sc-kv6pi1-0 dtSdUZ']")
    temp = []
    sleep(3)

    if features[0].text.split('\n')[0] == 'Languages spoken':
        features.pop(0)

    for i in range(len(features)):
        temp = temp + features[i].text.split('\n')[1:]

    sub_fac_df = pd.DataFrame({'hotel_ID': hotelID,
                          'property_feature':temp})
    return sub_fac_df

def getCustomerReviewsDataFrame():
    usernames = []
    user_rating = []

    hotelID = driver.find_element(By.XPATH, "//p[@class ='HeaderCerebrum__Name']").text
    user_info = driver.find_elements(By.XPATH, "//div[@data-info-type='reviewer-name']")
    ratings = driver.find_elements(By.XPATH, "//div[@class = 'Review-comment-leftScore']")
    sleep(6)
    for i in range(len(user_info)):
        usernames.append(unidecode(user_info[i].text.split()[0]).strip())
        user_rating.append(float(ratings[i].text))

    user_df = pd.DataFrame({
        'userID': usernames,
        'Rating':user_rating,
        'hotel_ID': hotelID
    })
    return user_df

In [45]:
features_df = pd.DataFrame(columns = ['hotel_ID', 'property_feature'])
users_df = pd.DataFrame(columns = ['userID', 'Rating'])

In [46]:
counter = 0

while counter < len(links):
    try:
        driver.get(links[counter])
        sleep(5)
        scrollDown()
        sleep(randint(3,7))
        features_df = pd.concat([features_df,getHotelFeaturesDataFrame()], ignore_index= True)
        users_df = pd.concat([users_df, getCustomerReviewsDataFrame()], ignore_index = True)
        
        counter = counter + 1
        
    except:
        print('Exception Occured')
        sleep(randint(60,80))
        driver.refresh()

Exception Occured


In [ ]:
users_df.to_csv('user_ratings.csv', index=False)
features_df.to_csv('hotel_features.csv', index=False)